In [1]:
# langchain-community
# langchain_openai -> langchain 중에서 openai랑 관련된 것
!pip install langchain openai langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.0 MB/s eta 0:00:00


In [2]:
import os, openai
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('sesac7_openAI_key')

In [3]:
# 모델의 응답을 '문자열'로 파싱하는 데 사용
# 언어 모델의 출력을 단순한 텍스트 형태로 변형
from langchain_core.output_parsers import StrOutputParser

# 대화형 프롬프트의 양식(프롬프트 템플릿)을 정의하는 클래스
from langchain_core.prompts import ChatPromptTemplate

# OpenAI의 언어 모델
from langchain_openai import ChatOpenAI

# {topic}이 input_variables로 분류됨.
template = '{topic}을 주제로 한 퀴즈를 내줘.'
chat_prompt_template = ChatPromptTemplate.from_template('{topic}을 주제로 한 퀴즈를 내줘.')

chat = ChatOpenAI(temperature=0.7)

output_parsers = StrOutputParser()

# chat_prompt_template -> chat -> output_parser
# 하나의 Chain으로 묶어주고 싶다.
# LCEL 문법 : | (파이프) chain을 |로 묶어서 정의해준다.
chain = chat_prompt_template | chat | output_parsers
chain.invoke({'topic':'코딩'})

'당신이 좋아하는 언어로 작성된 코드를 보고 그 코드가 무엇을 하는지 추측해보세요. \n\n```python\ndef is_prime(num):\n    if num < 2:\n        return False\n    for i in range(2, num):\n        if num % i == 0:\n            return False\n    return True\n\nprint(is_prime(17))\n```\n\n이 코드는 주어진 숫자가 소수인지 확인하는 함수를 정의하고, 17이 소수인지 확인하는 것을 출력하는 코드입니다. 결과는 True가 나올 것입니다.'

In [4]:
chain = chat_prompt_template | ChatOpenAI() | StrOutputParser()
chain.invoke({'topic':'여름'})

'1. 여름에 가장 많이 볼 수 있는 식물은 무엇인가요?\n2. 여름방학 때 가장 인기 있는 여행지는 어디인가요?\n3. 여름에 먹기 좋은 과일은 무엇인가요?\n4. 여름에 가장 좋아하는 레저 활동은 무엇인가요?\n5. 여름에 필수적으로 가져다닐 것은 무엇인가요?'

## 러너블(Runnable)

- runnable passThrough : 입력값을 -> 다음의 출력값으로 전달하는 러너블. (디폴트 형태)
- runnable Parallel : 여러 개의 러너블 객체를 병렬로 실행
- runnable Lambda : 임의의 함수를 러너블로 감싸서 활용할 수 있게 함

In [6]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

chat_prompt_template = ChatPromptTemplate.from_template('{topic}에 대한 유용한 정보를 알려줘.')
chain = {'topic': RunnablePassthrough()} | chat_prompt_template | ChatOpenAI() | StrOutputParser()
chain.invoke('몬헌 와일즈 수렵피리')

'몬헌 와일즈 수렵피리는 몬스터 헌터 시리즈에서 사용되는 아이템으로, 몬스터를 유인하거나 방해하는 효과를 가지고 있습니다.\n\n이 아이템은 주로 큰 영역에 존재하는 몬스터를 유인하여 사냥하기 쉽게 만들어줍니다. 또한, 몬스터간의 전투를 유도하여 자신의 이익을 얻을 수도 있습니다.\n\n수렵피리는 다양한 유형이 있으며, 각각의 수렵피리는 다른 효과와 사용 방법을 가지고 있습니다. 플레이어는 상황에 따라 적절한 수렵피리를 선택하여 사용해야 합니다.\n\n몬헌 와일즈 수렵피리는 전투와 사냥을 보다 효율적으로 진행할 수 있도록 도와주는 유용한 아이템이므로, 적극적으로 활용해보는 것을 권장합니다.'

## 러너블 클래스를 정의하여 함수 감싸기

https://medium.com/@MichaelHashimoto/why-the-pipe-character-works-in-langchains-lcel-b4e8685855f5


In [7]:
# args(arguments) -> 위치를 기반으로 매개변수 매칭 => 튜플로 받아짐
# kwargs(keyword arguments) -> 키워드를 기반으로 매개변수 매칭 => 딕셔너리로 받아짐
def test(*args, **kwargs):
    print('args', args)
    print('kwargs', kwargs)
test(1, 2, 1, 3, 4, 'd', name='julia', fruit='watermelon')

args (1, 2, 1, 3, 4, 'd')
kwargs {'name': 'julia', 'fruit': 'watermelon'}


In [8]:
class Runnable:
    # 함수를 매개변수(func)로 받아서, 함수의 동작을 변형
    def __init__(self, func):
        self.func = func

    # __or__ == | < 새로운 함수로 정의(오버라이딩)
    # | 기호가 어떻게 동작할지를 새롭게 알려줌
    # A(func) | B(other)
    def __or__(self, other):
        def chained_function(*args, **kwargs):
            result = self.func(*args, **kwargs)
            return other(result)
        return Runnable(chained_function)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [9]:
def add_five(x):
    return x + 5

def multiple_three(x):
    return x * 3

In [11]:
add_five = Runnable(add_five)
multiple_three = Runnable(multiple_three)

add_five

In [12]:
chain = add_five | multiple_three
chain(5)

30

In [14]:
def explain(x):
    print(f"계산한 결과는 {x}입니다.")

chain = add_five | multiple_three | explain
chain(5)

계산한 결과는 30입니다.
